In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import matplotlib.pyplot as plt
import sys

In [2]:
input_fname = 'test_toymodel.in'
np.set_printoptions(formatter={'float': lambda x: format(x, '1.2E')})
nTurn = 1024
nTurnSplit = 0
dTune = 0
nCore_y = 1
nCore_z = 1

In [3]:
# old_stdout = sys.stdout
# log_file   = open("log."+str(dTune),"w")
# sys.stdout = log_file

In [4]:
def getArgLostLive(TBTdata,rad):
    dummy,iLost = np.where(np.sqrt(TBTdata[:,0,:]**2+TBTdata[:,2,:]**2)>rad)
    iLost = set(iLost)
    iLive = set(np.arange(npt))
    iLive = iLive - iLost
    iLost=np.array(list(iLost))
    iLive=np.array(list(iLive))
    return iLost,iLive

##### grid data for contour plot

In [5]:
from scipy.interpolate import griddata
xmax = 0.01
xi = np.linspace(-xmax*1.1,xmax*1.1,240)
yi = np.linspace(-xmax*1.1,xmax*1.1,240)

# 1. Preparation

### 1.1 read InputFile and simplify lattice

In [6]:
beam, lattice = impact.readInputFile(input_fname);
beam.nCore_y = nCore_y
beam.nCore_z = nCore_z
beam.distribution.distribution_type = 'ReadFile'
beam.current = 0.0
lattice[0].turns = 2*nTurn + nTurnSplit
betx = beam.distribution.betx
alfx = beam.distribution.alfx

reading ImpactZ input file (test_toymodel.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [7]:
for i in range(len(lattice)-1,0,-1):
    if lattice[i].type == 'write_raw_ptcl':
        lattice.pop(i)
lattice[1].n_sckick = 100
lattice[2].tune_advance = 0.01*dTune

### 1.2. generate particles

In [8]:
testP = 0.0
k=0
q_m = beam.multi_charge.q_m[0]
pTest = []
x=0.002
y=0.002
px=0.002
py=0.002

pTest.append([x, px,y, py,0.0,0.0,q_m,0.0,13])
pTest.append([x,-px,y, py,0.0,0.0,q_m,0.0,14])
pTest.append([x, px,y,-py,0.0,0.0,q_m,0.0,15])
pTest.append([x,-px,y,-py,0.0,0.0,q_m,0.0,16])
pTest.append([-x, px,y, py,0.0,0.0,q_m,0.0,13])
pTest.append([-x,-px,y, py,0.0,0.0,q_m,0.0,14])
pTest.append([-x, px,y,-py,0.0,0.0,q_m,0.0,15])
pTest.append([-x,-px,y,-py,0.0,0.0,q_m,0.0,16])
pTest.append([x, px,-y, py,0.0,0.0,q_m,0.0,13])
pTest.append([x,-px,-y, py,0.0,0.0,q_m,0.0,14])
pTest.append([x, px,-y,-py,0.0,0.0,q_m,0.0,15])
pTest.append([x,-px,-y,-py,0.0,0.0,q_m,0.0,16])
pTest.append([-x, px,-y, py,0.0,0.0,q_m,0.0,13])
pTest.append([-x,-px,-y, py,0.0,0.0,q_m,0.0,14])
pTest.append([-x, px,-y,-py,0.0,0.0,q_m,0.0,15])
pTest.append([-x,-px,-y,-py,0.0,0.0,q_m,0.0,16])

for i in range(len(pTest)):
    pTest[i][-1]=i+1

pTest = np.array(pTest)
ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency
impact.writeParticleData(pTest, ke, mass, freq)

npt = len(pTest)
beam.n_particles = npt
print('initial npt=',npt)

initial npt= 16


# 2. FMA

### 2.1. run

In [9]:
print('===========================')
print('=========== FMA ===========')
print('===========================')

=========== FMA ===========


In [10]:
TBT = impact.getElem('TBT')
TBT.file_id = dTune
lattice.insert(1,TBT)

TBT_Inv = impact.getElem('TBT_integral')
TBT_Inv.file_id = dTune
TBT_Inv.betx = betx
TBT_Inv.alfx = alfx
TBT_Inv.strength_t = 0.4
TBT_Inv.transverse_scale_c = 0.01
lattice.insert(2,TBT_Inv)

impact.writeInputFile(beam,lattice) 
impact.run(beam)

0

In [11]:
indexTBT,TBTdata = impact.readTBT(dTune,ke,mass,freq)

In [12]:
X = TBTdata[:,0,:] - 1j*(betx*TBTdata[:,1,:]+alfx*TBTdata[:,0,:])
Y = TBTdata[:,2,:] - 1j*(betx*TBTdata[:,3,:]+alfx*TBTdata[:,2,:])
tuneXbefore= np.zeros([1,npt])
tuneXafter = np.zeros([3,npt])
tuneYbefore= np.zeros([1,npt])
tuneYafter = np.zeros([3,npt])
for i in range(npt):
    tuneXbefore[:,i],amp,dummy = impact.util.naff(1,X[:nTurn,i],window_id=1)
    tuneXafter [:,i],amp,dummy = impact.util.naff(3,X[nTurn+nTurnSplit:,i],window_id=1)
    tuneYbefore[:,i],amp,dummy = impact.util.naff(1,Y[:nTurn,i],window_id=1)
    tuneYafter [:,i],amp,dummy = impact.util.naff(3,Y[nTurn+nTurnSplit:,i],window_id=1)

In [13]:
def getClosestDiff(ref,arr):
    # ref.shape = nSample
    nMode, nSample = arr.shape
    diff = np.abs(ref-arr)
    for i in range(nMode):
        for j in range(nSample):
            if (diff[i,j]>0.5):
                diff[i,j] = 1.0 - diff[i,j]
    return np.min(diff,0)

In [14]:
Xdiff = getClosestDiff(tuneXbefore,tuneXafter)
Ydiff = getClosestDiff(tuneYbefore,tuneYafter)
Adiff = np.sqrt(Xdiff**2 +Ydiff**2)

Xstd = np.std(Xdiff)
Ystd = np.std(Ydiff)
Astd = np.sqrt(Xstd**2+Ystd**2)

In [15]:
print(Adiff)

[3.28E-12 6.35E-14 5.93E-11 1.86E-12 6.35E-14 3.28E-12 1.86E-12 5.93E-11
 5.93E-11 1.86E-12 3.28E-12 6.35E-14 1.86E-12 5.93E-11 6.35E-14 3.28E-12]


In [16]:
for i in range(len(X[0,:])):
    print(X[0,i],Y[0,i])

(0.0019999999443449314-0.006629462926754244j) (0.0019999999443449314-0.006629462926754244j)
(0.0019999999443449314+0.0009965586551566358j) (0.0019999999443449314-0.006629462926754244j)
(0.0019999999443449314-0.006629462926754244j) (0.0019999999443449314+0.0009965586551566358j)
(0.0019999999443449314+0.0009965586551566358j) (0.0019999999443449314+0.0009965586551566358j)
(-0.0019999999443449314-0.0009965586551566358j) (0.0019999999443449314-0.006629462926754244j)
(-0.0019999999443449314+0.006629462926754244j) (0.0019999999443449314-0.006629462926754244j)
(-0.0019999999443449314-0.0009965586551566358j) (0.0019999999443449314+0.0009965586551566358j)
(-0.0019999999443449314+0.006629462926754244j) (0.0019999999443449314+0.0009965586551566358j)
(0.0019999999443449314-0.006629462926754244j) (-0.0019999999443449314-0.0009965586551566358j)
(0.0019999999443449314+0.0009965586551566358j) (-0.0019999999443449314-0.0009965586551566358j)
(0.0019999999443449314-0.006629462926754244j) (-0.0019999999443